In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd

In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/src/src/static/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import requests
from bs4 import BeautifulSoup
import csv
import time
import os

In [3]:
# Directorios para Kaggle
OUTPUT_DIR = "src/src/static/kaggle"

In [4]:
# URLs base para el scraping
BASE_URL_BALOTO = "https://www.baloto.com/resultados-baloto/"
BASE_URL_REVANCHA = "https://www.baloto.com/resultados-revancha/"

In [5]:
# Nombres de los archivos CSV de salida
OUTPUT_FILE_BALOTO = os.path.join(OUTPUT_DIR, "baloto.csv")
OUTPUT_FILE_REVANCHA = os.path.join(OUTPUT_DIR, "revancha.csv")

In [6]:
os.makedirs("src/src/static/kaggle", exist_ok=True)

In [7]:
for output_file in [OUTPUT_FILE_BALOTO, OUTPUT_FILE_REVANCHA]:
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "C1", "C2", "C3", "C4", "C5", "SB", "#Sorteo"])


In [8]:
# Función para extraer información de un sorteo específico
def scrape_sorteo(url_base, output_file, sorteo_id, retries=3):
    url = f"{url_base}{sorteo_id}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    attempt = 0
    while attempt < retries:
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            # Extraer los datos del sorteo
            sorteo_tag = soup.find("div", class_="gotham-medium dark-blue fs-5")
            numero_sorteo = sorteo_tag.find("strong").text.strip().replace("SORTEO ", "") if sorteo_tag else "Número no disponible"

            fecha_tag = soup.find("div", class_="gotham-medium dark-blue")
            fecha = fecha_tag.text.strip() if fecha_tag else "Fecha no disponible"

            balotas_container = soup.find("div", class_="container-balls-results")
            if balotas_container:
                balotas = [
                    ball.text.strip() for ball in balotas_container.find_all("div", class_="yellow-ball gotham-medium")
                ]
                superbalota_tag = balotas_container.find("div", class_="red-ball gotham-medium")
                superbalota = superbalota_tag.text.strip() if superbalota_tag else "SB no disponible"

                if len(balotas) >= 5:
                    balota1, balota2, balota3, balota4, balota5 = balotas[:5]

                    # Guardar los datos en el archivo CSV
                    with open(output_file, mode="a", newline="", encoding="utf-8") as file:
                        writer = csv.writer(file)
                        writer.writerow([fecha, balota1, balota2, balota3, balota4, balota5, superbalota, numero_sorteo])
                    print(f"Sorteo {numero_sorteo} extraído y guardado en {output_file}.")
                    return
                else:
                    print(f"Datos incompletos para el sorteo {numero_sorteo}.")
                    return
            else:
                print(f"No se encontró el contenedor de balotas para el sorteo {sorteo_id}.")
                return
        except requests.exceptions.RequestException as e:
            attempt += 1
            if attempt < retries:
                print(f"Error de conexión para el sorteo {sorteo_id}. Reintentando ({attempt}/{retries})...")
                time.sleep(5)  # Esperar antes de reintentar
            else:
                print(f"Error de conexión para el sorteo {sorteo_id}. No se pudo recuperar después de {retries} intentos: {e}")
                return
        except Exception as e:
            print(f"Error procesando el sorteo {sorteo_id}: {e}")
            return


In [9]:
# Iterar sobre un rango de sorteos para Baloto
START_SORTEO = 2491  # Cambiar según sea necesario
END_SORTEO = 2460    # Cambiar según sea necesario

for sorteo_id in range(START_SORTEO, END_SORTEO - 1, -1):
    scrape_sorteo(BASE_URL_BALOTO, OUTPUT_FILE_BALOTO, sorteo_id)
    scrape_sorteo(BASE_URL_REVANCHA, OUTPUT_FILE_REVANCHA, sorteo_id)
    time.sleep(1)  # Pausa para evitar sobrecargar el servidor

Sorteo 2.491 extraído y guardado en src/src/static/kaggle\baloto.csv.
Sorteo 2.491 extraído y guardado en src/src/static/kaggle\revancha.csv.
Sorteo 2.490 extraído y guardado en src/src/static/kaggle\baloto.csv.
Sorteo 2.490 extraído y guardado en src/src/static/kaggle\revancha.csv.
Sorteo 2.489 extraído y guardado en src/src/static/kaggle\baloto.csv.
Sorteo 2.489 extraído y guardado en src/src/static/kaggle\revancha.csv.
Sorteo 2.488 extraído y guardado en src/src/static/kaggle\baloto.csv.
Sorteo 2.488 extraído y guardado en src/src/static/kaggle\revancha.csv.
Sorteo 2.487 extraído y guardado en src/src/static/kaggle\baloto.csv.
Sorteo 2.487 extraído y guardado en src/src/static/kaggle\revancha.csv.
Sorteo 2.486 extraído y guardado en src/src/static/kaggle\baloto.csv.
Sorteo 2.486 extraído y guardado en src/src/static/kaggle\revancha.csv.
Sorteo 2.485 extraído y guardado en src/src/static/kaggle\baloto.csv.
Sorteo 2.485 extraído y guardado en src/src/static/kaggle\revancha.csv.
Sorteo